In [6]:
import sys
sys.path.append('../')
from model.VQAE import VQAE
from model.MARL import MARL
from utils import device, add_noise
from tqdm import tqdm
import torch
from data.FloorPlanLoader import *
import torch.nn.functional as F
import random
import json
import torchvision
import matplotlib.pyplot as plt
import geopandas as gpd

USE_MULTISCALE = True
USE_MULTITASK = True

#Reproducability Checks:
random.seed(0) #Python
torch.manual_seed(0) #Torch
np.random.seed(0) #NumPy

import shutil
import torchvision
import torchvision.transforms.functional as TF
import imageio
from torchvision.utils import save_image
# %load_ext autoreload
# %autoreload 2
import csv
import numpy
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import pandas as pd
import seaborn as sns
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [2]:
#Hyperparameter
device = torch.device('cpu')
batch_size = 128
n_hiddens = 32
n_residual_hiddens = 32
n_residual_layers = 1
embedding_dim = 64
n_embeddings = 218
beta = .25
lr = 3e-3
epochs = 100
noise=False
noise_weight=0.05
img_channel=3 if USE_MULTISCALE else 1

In [16]:
curr_data = 'data20_left'
n_clusters= 5

latent_file_path = f'../data/data_root/marl_latent_{curr_data}.pt'
latents = torch.load(latent_file_path)
floor = FloorPlanDataset(multi_scale=True, root=f'../data/data_root/{curr_data}/',\
                         data_config='../data/data_config/', preprocess=True)
data_loader = torch.utils.data.DataLoader(floor, batch_size=batch_size, shuffle=False)

In [ ]:
import geopandas as gpd

In [13]:

# Replace 'path_to_json_file.json' with the actual path to your JSON file.
gdf = gpd.read_file('../data/data_root/geojson_80.json')

In [14]:
gdf.head(2)

,OBJECTID,CODE,BLD_ID,YearBuilt1,HEIGHT,ELEV,footprint,UseType,UseDescription,Shape_Length_1,Shape_Area_1,geometry
0,2099602,Building,447500864349,1924,29.50,975.64,2587.0,Residential,Single,1256.638268,18764.3232,"MULTIPOLYGON (((-118.37715 34.11478, -118.3771..."
1,2099603,Building,445742863736,1958,10.06,1032.63,207.0,Residential,Single,301.438567,5069.7824,"MULTIPOLYGON (((-118.38298 34.11306, -118.3829..."


In [15]:
# Assuming the 'OBJECTID' column exists in the GeoDataFrame 'gdf'.
objectid_df = gdf[['OBJECTID']]
objectid_df.head(2)

,OBJECTID
0,2099602
1,2099603


In [17]:
df_center_index = pd.read_csv(f'../results/MARL_{curr_data}_{n_clusters}clusters_clusterindex.csv', index_col=None)
df_center_index

,label,index
0,3,523381
1,2,523382
2,2,523383
3,0,523384
4,0,523385
...,...,...
13326,3,549727
13327,0,549728
13328,1,549729
13329,2,549730


In [11]:
df_center_index_VQAE = pd.read_csv(f'../results/VQAE_{curr_data}_{n_clusters}clusters_clusterindex.csv', index_col=None)
df_center_index_VQAE

,label,index
0,0,1000
1,1,1001
2,4,1006
3,0,1007
4,4,1008
...,...,...
13972,0,993
13973,2,996
13974,1,997
13975,2,998


In [7]:
metadata_file_path = r"..\data\data_root\building_metadata_clean.csv"
metadata_df = pd.read_csv(metadata_file_path, index_col=False)
metadata_df.head(5)

,OBJECTID,vintage,idf.kw,FootprintArea.m2,building.area.m2,HEIGHT
0,19,1981-2008 (2004),SingleFamily-2004,302.896371,315.9529,22.87
1,20,1981-2008 (2004),SingleFamily-2004,91.815805,153.5637,25.52
2,21,before 1980,SingleFamily-pre-1980,176.129465,200.8498,22.21
3,22,before 1980,SingleFamily-pre-1980,113.052927,153.0992,28.80
4,23,1981-2008 (2004),SingleFamily-2004,108.517313,152.4489,22.32


In [16]:
merged_df = pd.merge(objectid_df, metadata_df, left_on='OBJECTID', right_on = 'OBJECTID', how='inner')
print(objectid_df.shape, merged_df.shape)
merged_df

(22611, 1) (9875, 6)


,OBJECTID,vintage,idf.kw,FootprintArea.m2,building.area.m2,HEIGHT
0,2099603,1981-2008 (2004),SingleFamily-2004,307.331319,217.5718,18.11
1,2099604,1981-2008 (2004),SingleFamily-2004,158.310718,199.3634,26.45
2,2099606,before 1980,SingleFamily-pre-1980,232.630210,150.3122,16.71
3,2099607,1981-2008 (2004),SingleFamily-2004,320.681808,317.5322,25.58
4,2099608,before 1980,SingleFamily-pre-1980,211.345775,137.9565,18.18
...,...,...,...,...,...,...
9870,2125016,before 1980,SingleFamily-pre-1980,148.868437,86.0254,NaN
9871,2125019,before 1980,MultiFamily-pre-1980,334.785458,470.1669,NaN
9872,2125023,1981-2008 (2004),MultiFamily-2004,339.881529,493.1132,NaN
9873,2125025,before 1980,MultiFamily-pre-1980,90.716628,601.4346,12.33


In [11]:
merged_df['idf.kw'].unique()

array(['SingleFamily-pre-1980', 'SingleFamily-2004',
       'MultiFamily-pre-1980', 'MultiFamily-2004', 'SingleFamily-2013',
       'MultiFamily-2013'], dtype=object)

In [12]:
# geojson 4
# Separate 'SingleFamily' values
single_family_df = merged_df[merged_df['idf.kw'].str.contains('SingleFamily')]
print(single_family_df.shape)
# Separate 'MultiFamily' values
multi_family_df = merged_df[merged_df['idf.kw'].str.contains('MultiFamily')]
print(multi_family_df.shape)

(11419, 6)
(348, 6)


In [17]:
# 8 
# Separate 'SingleFamily' values
single_family_df = merged_df[merged_df['idf.kw'].str.contains('SingleFamily')]
print(single_family_df.shape)
# Separate 'MultiFamily' values
multi_family_df = merged_df[merged_df['idf.kw'].str.contains('MultiFamily')]
print(multi_family_df.shape)

(7651, 6)
(2224, 6)


In [22]:
single_family_df[['index']].to_csv(f'../notebooks/categorical_clustering/{curr_data}_singlefamily.csv')
multi_family_df[['index']].to_csv(f'../notebooks/categorical_clustering/{curr_data}_multifamily.csv')

In [25]:
csv_file = f'../notebooks/categorical_clustering/{curr_data}_multifamily.csv'
# Read the CSV file into a dataframe
df = pd.read_csv(csv_file)
df

,Unnamed: 0,index
0,13,523490
1,15,523496
2,18,523502
3,20,523505
4,21,523508
...,...,...
1417,6926,549596
1418,6935,549658
1419,6941,549677
1420,6944,549680


In [27]:

# Assuming your CSV file has a column named 'OBJECTID' and the images are in the format 'f{objectid}.png'
curr_data_folder = f'../data/data_root/{curr_data}_SF/'
output_folder = f'../data/data_root/{curr_data}_MF/'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
num_files_copied = 0

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    object_id = row['index']
    image_file = f'{object_id}.png'
    pt_file = f'{object_id}.pt'
    img_source_path = os.path.join(curr_data_folder, image_file)
    img_destination_path = os.path.join(output_folder, image_file)
    
    pt_source_path = os.path.join(curr_data_folder, pt_file)
    pt_destination_path = os.path.join(output_folder, pt_file)
    
    if os.path.exists(img_source_path):
        shutil.move(img_source_path, img_destination_path)
        shutil.move(pt_source_path, pt_destination_path)
        num_files_copied += 1

# Display the total number of files copied
print(f"Total number of files copied: {num_files_copied}")

Total number of files copied: 1422
